In [1]:
#%cd /content/drive/My Drive/SegFormer
import argparse
import logging
import os
import random
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from utils import clip_gradient, AvgMeter

from glob import glob
from skimage.io import imread
import matplotlib.pyplot as plt
import pandas as pd
from collections import OrderedDict
from torch.autograd import Variable
from datetime import datetime
import torch.nn.functional as F
import cv2
from albumentations.augmentations import transforms
from albumentations.core.composition import Compose, OneOf
import math

/home/s/anaconda3/envs/tf2.6/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import argparse
import copy
import os
import os.path as osp
import time

import mmcv
import torch
from mmcv.runner import init_dist
from mmcv.utils import Config, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

In [3]:
from mmseg.models.segmentors import CaraSegUPer_ver2 as UNet
from mmseg.models.segmentors import CaraSegUPer_wBiFPN as Net
from mmseg.models.segmentors import CaraSegUPer_wBiRAFPN as Net2

In [4]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, img_paths, mask_paths, aug=True, transform=None):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.aug = aug
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        mask_path = self.mask_paths[idx]
        # image = imread(img_path)
        # mask = imread(mask_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, 0)
        # name = self.img_paths[idx].split('/')[-1]

        if self.transform is not None:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 
        else:
            image = cv2.resize(image, (384, 384))
            mask = cv2.resize(mask, (384, 384)) 

        image = image.astype('float32') / 255
        image = image.transpose((2, 0, 1))

        mask = mask[:,:,np.newaxis]
        mask = mask.astype('float32') / 255
        mask = mask.transpose((2, 0, 1))

        return np.asarray(image), np.asarray(mask)

In [5]:
from keras import backend as K

def recall_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  possible_positives = torch.sum(torch.round(torch.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = torch.sum(torch.round(torch.clip(y_true * y_pred, 0, 1)))
  predicted_positives = torch.sum(torch.round(torch.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def dice_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return recall*precision/(recall+precision-recall*precision +K.epsilon())

In [6]:
class FocalLossV1(nn.Module):

    def __init__(self,
                 alpha=0.25,
                 gamma=2,
                 reduction='mean',):
        super(FocalLossV1, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.crit = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, label):
        # compute loss
        logits = logits.float() # use fp32 if logits is fp16
        with torch.no_grad():
            alpha = torch.empty_like(logits).fill_(1 - self.alpha)
            alpha[label == 1] = self.alpha

        probs = torch.sigmoid(logits)
        pt = torch.where(label == 1, probs, 1 - probs)
        ce_loss = self.crit(logits, label.float())
        loss = (alpha * torch.pow(1 - pt, self.gamma) * ce_loss)
        if self.reduction == 'mean':
            loss = loss.mean()
        if self.reduction == 'sum':
            loss = loss.sum()
        return loss
class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, eps=1):
        super().__init__()

        self.eps = eps
        self.alpha = alpha

    def forward(self, pred, label):
        """
        Forward function
        :param pred: Prediction tensor containing raw network outputs (no logit) (B x C x H x W)
        :param label: Label mask tensor (B x C x H x W)
        """
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])
        return 1 - torch.mean(
            (true_pos + self.eps)
            / (
                true_pos
                + self.alpha * false_neg
                + (1 - self.alpha) * false_pos
                + self.eps
            )
        )

class FocalTverskyLoss(TverskyLoss):
    def __init__(self, gamma=4 / 3, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma

    def forward(self, pred, label):
        probs = torch.sigmoid(pred)
        true_pos = torch.sum(probs * label, dim=[0, 2, 3])
        false_neg = torch.sum(label * (1 - probs), dim=[0, 2, 3])
        false_pos = torch.sum(probs * (1 - label), dim=[0, 2, 3])

        t = (true_pos + self.eps) / (
            true_pos + self.alpha * false_neg + (1 - self.alpha) * false_pos + self.eps
        )

        x = torch.pow(1 - t, 1 / self.gamma)

        return x #torch.sum(x)
def structure_loss_v2(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))

    
    wiou = FocalTverskyLoss()(pred,mask)
    return (wfocal + wiou).mean()
def structure_loss(pred, mask):
    weit = 1 + 5*torch.abs(F.avg_pool2d(mask, kernel_size=31, stride=1, padding=15) - mask)
    wfocal = FocalLossV1()(pred, mask)
    wfocal = (wfocal*weit).sum(dim=(2,3)) / weit.sum(dim=(2, 3))

    pred = torch.sigmoid(pred)
    inter = ((pred * mask)*weit).sum(dim=(2, 3))
    union = ((pred + mask)*weit).sum(dim=(2, 3))
    wiou = 1 - (inter + 1)/(union - inter+1)
    return (wfocal + wiou).mean()

In [7]:
def train(train_loader, model, optimizer, epoch, lr_scheduler, deep=False):
    model.train()
    # ---- multi-scale training ----
    size_rates = [256/384, 1, 512/384]
    loss_record = AvgMeter()
    dice, iou = AvgMeter(), AvgMeter()
    for i, pack in enumerate(train_loader, start=1):
        if epoch <= 1:
                optimizer.param_groups[0]["lr"] = (epoch * i) / (1.0 * total_step) * init_lr
        else:
            lr_scheduler.step()

        for rate in size_rates: 
            optimizer.zero_grad()
            # ---- data prepare ----
            images, gts = pack
            images = Variable(images).cuda()
            gts = Variable(gts).cuda()
            # ---- rescale ----
            trainsize = int(round(trainsize_init*rate/32)*32)
            images = F.upsample(images, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            gts = F.upsample(gts, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            # ---- forward ----
            map4, map3, map2, map1 = model(images)
           # print(map4.shape, map3.shape, map2.shape, map1.shape)
            map1 = F.upsample(map1, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map2 = F.upsample(map2, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map3 = F.upsample(map3, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            map4 = F.upsample(map4, size=(trainsize, trainsize), mode='bilinear', align_corners=True)
            loss = structure_loss(map1, gts) + structure_loss(map2, gts) + structure_loss(map3, gts) + structure_loss(map4, gts)
            # ---- metrics ----
            dice_score = dice_m(map2, gts)
            iou_score = iou_m(map2, gts)
            # ---- backward ----
            loss.backward()
            clip_gradient(optimizer, clip)
            optimizer.step()
            # ---- recording loss ----
            if rate == 1:
                loss_record.update(loss.data, batchsize)
                dice.update(dice_score.data, batchsize)
                iou.update(iou_score.data, batchsize)

        # ---- train visualization ----
        if i == total_step:
            print('{} Training Epoch [{:03d}/{:03d}], '
                  '[loss: {:0.4f}, dice: {:0.4f}, iou: {:0.4f}]'.
                  format(datetime.now(), epoch, num_epochs,\
                         loss_record.show(), dice.show(), iou.show()))

    ckpt_path = save_path + 'last.pth'
    print('[Saving Checkpoint:]', ckpt_path)
    checkpoint = {
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': lr_scheduler.state_dict()
    }
    torch.save(checkpoint, ckpt_path)

    log = OrderedDict([
        ('loss', loss_record.show()), ('dice', dice.show()), ('iou', iou.show()),
    ])

    return log

In [11]:
def recall_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_np(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def dice_np(y_true, y_pred):
    precision = precision_np(y_true, y_pred)
    recall = recall_np(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def iou_np(y_true, y_pred):
    intersection = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    union = np.sum(y_true)+np.sum(y_pred)-intersection
    return intersection/(union+K.epsilon())

def acc_np(y_true, y_pred):
    acc = np.sum(y_true==y_pred)
    h,w = y_true.shape
    return acc/(h*w+K.epsilon())

def get_scores(gts, prs):
    mean_precision = 0
    mean_recall = 0
    mean_iou = 0
    mean_dice = 0
    mean_acc = 0
    for gt, pr in zip(gts, prs):
        mean_precision += precision_np(gt, pr)
        mean_recall += recall_np(gt, pr)
        mean_iou += iou_np(gt, pr)
        mean_dice += dice_np(gt, pr)
        mean_acc += acc_np(gt, pr)

    mean_precision /= len(gts)
    mean_recall /= len(gts)
    mean_iou /= len(gts)
    mean_dice /= len(gts)        
    mean_acc /= len(gts) 
    print(f"scores: dice={mean_dice}, miou={mean_iou}, precision={mean_precision}, recall={mean_recall}, accuracy ={mean_acc}")

    return (mean_iou, mean_dice, mean_precision, mean_recall)



def inference(model,writer=None,epoch=None):
    print("#"*20)
    model.eval()
    dataset_names = ['ISIC']
    for dataset_name in dataset_names:
        data_path = f'../dataset/ISIC/384/testdataset/'
        print(data_path)
        X_test = glob('{}/images/*'.format(data_path))
        X_test.sort()
        y_test = glob('{}/masks/*'.format(data_path))
        y_test.sort()

        test_dataset = Dataset(X_test, y_test)
        test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=1,
            shuffle=False,
            pin_memory=True,
            drop_last=False)

        print('Dataset_name:', dataset_name)
        tp_all = 0
        fp_all = 0
        fn_all = 0
        mean_iou = 0
        gts = []
        prs = []
        losses = []
        for i, pack in enumerate(test_loader, start=1):
            image, gt_ = pack
            # name = name[0]
            gt = gt_[0][0]
            gt = np.asarray(gt, np.float32)
            image = image.cuda()

            res, res2, res3, res4 = model(image)
            res = F.upsample(res, size=gt.shape, mode='bilinear', align_corners=False)
            loss = structure_loss_v2(res.cpu(), gt_)
            losses.append(loss.cpu().detach().numpy().squeeze())
            res = res.sigmoid().data.cpu().numpy().squeeze()
            res = (res - res.min()) / (res.max() - res.min() + 1e-8)
            pr = res.round()
            # cv2.imwrite(os.path.join(save_path, dataset_name, name), res)
            gts.append(gt)
            prs.append(pr)
        mean_iou, mean_dice, mean_precision, mean_recall = get_scores(gts, prs)
#         writer.add_scalar(dataset_name + ' val loss',
#                             np.mean(losses),
#                             epoch )
#         writer.add_scalar(dataset_name + ' val dice',
#                             mean_dice,
#                             epoch )
#         writer.add_scalar(dataset_name + ' val iou',
#                             mean_iou,
#                             epoch )
    print("#"*20)

In [12]:
#from torchvision.transforms.functional import *
#import torchvision.transforms as transforms
from albumentations.augmentations.geometric import  resize,rotate
import albumentations.augmentations.crops.transforms as crop
import albumentations.augmentations.transforms as transforms
train_transform = Compose([
            rotate.RandomRotate90(),
            transforms.Flip(),
            transforms.HueSaturationValue(),
            transforms.RandomBrightnessContrast(),
            transforms.GaussianBlur(),
            transforms.Transpose(),
            
#             resize.Resize(384, 384),
#     OneOf([
#                 crop.RandomCrop(224, 224, p=1),
#                 crop.CenterCrop(224, 224, p=1)
#             ], p=0.2),
    resize.Resize(384, 384),
        ], p=0.5)

In [13]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsISIC_RA/lan2')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 50
train_save = 'BiRAFPN_ISIC_2'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/ISIC/384/traindataset/images/*')#+glob('../dataset/ISIC/384/valdataset/images/*')
train_mask_paths = glob('../dataset/ISIC/384/traindataset/masks/*')#+glob('../dataset/ISIC/384/valdataset/masks/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)#train_transform
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                   compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()
model.load_state_dict(torch.load(save_path+"last.pth")['state_dict'])
inference(model,None,None)
"""
# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    #inference(model,writer,epoch)
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)
"""

Save path existed


2022-07-09 20:55:30,682 - mmseg - INFO - Use load_from_local loader
2022-07-09 20:55:30,767 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



####################
../dataset/ISIC/384/testdataset/
Dataset_name: ISIC
scores: dice=0.9039473368139728, miou=0.8397185246272749, precision=0.9195016311808233, recall=0.9145201417698453, accuracy =0.9633764674514371
####################


'\n# ---- flops and params ----\nparams = model.parameters()\noptimizer = torch.optim.Adam(params, init_lr)\nlr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, \n                                    T_max=len(train_loader)*num_epochs,\n                                    eta_min=init_lr/1000)\n\n\n\nstart_epoch = 1\n\nckpt_path = \'\'\nif ckpt_path != \'\':\n    log = pd.read_csv(ckpt_path.replace(\'last.pth\', \'log.csv\'))\n    checkpoint = torch.load(ckpt_path)\n    start_epoch = checkpoint[\'epoch\']\n    model.load_state_dict(checkpoint[\'state_dict\'])\n    lr_scheduler.load_state_dict(checkpoint[\'scheduler\'])\n    optimizer.load_state_dict(checkpoint[\'optimizer\'])\n\nprint("#"*20, f"Start Training", "#"*20)\nfor epoch in range(start_epoch, num_epochs+1):\n    #inference(model,writer,epoch)\n    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)\n\n    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], \n            train_log[\'l

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_v2/lan2')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_2_new_bottleneck_v2'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_v2/lan6')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_3_new_bottleneck_v2'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

In [13]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_v2/lan4')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_4_new_bottleneck_v2'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

2022-06-03 07:08:32,382 - mmseg - INFO - Use load_from_local loader
2022-06-03 07:08:32,453 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-06-03 07:11:56.016364 Training Epoch [001/030], [loss: 2.5475, dice: 0.7424, iou: 0.5989]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_v2/last.pth
2022-06-03 07:15:20.478696 Training Epoch [002/030], [loss: 1.7704, dice: 0.8021, iou: 0.6816]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_v2/last.pth
2022-06-03 07:18:44.814011 Training Epoch [003/030], [loss: 1.4272, dice: 0.8472, iou: 0.7412]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_v2/last.pth
2022-06-03 07:22:09.572724 Training Epoch [004/030], [loss: 1.2520, dice: 0.8630, iou: 0.7705]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_v2/last.pth
2022-06-03 07:25:34.102038 Training Epoch [005/030], [loss: 0.9965, dice: 0.8994, iou: 0.8213]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_v2/last.pth
2022-06-03 07:28:58.845722 Training Epoch [006/030], [loss: 1.0928, dice: 0.8947, iou: 0.8161]
[Saving Checkpoint:] ./

scores: dice=0.8036502064272391, miou=0.7253331694231795, precision=0.8152958649825075, recall=0.8476096854194509
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.8928240770457551, miou=0.8245918090382103, precision=0.8535267393915068, recall=0.9614237497729725
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.7628954278364037, miou=0.6854382734345537, precision=0.7143036219521227, recall=0.895849203716639
####################
2022-06-03 08:09:29.657805 Training Epoch [016/030], [loss: 0.7240, dice: 0.9467, iou: 0.9004]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_v2/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.917116598630243, miou=0.8643711960060009, precision=0.952044249622673, recall=0.9037668664532167
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB


scores: dice=0.9203907601264278, miou=0.8730226124994638, precision=0.9167185710855976, recall=0.9497457818820947
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.827606348510351, miou=0.7503771734044061, precision=0.8373141474475685, recall=0.8720441017585846
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.897959322151738, miou=0.8345714773272178, precision=0.8660579451615023, recall=0.9595578085487164
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.8082520277791204, miou=0.7282000852942342, precision=0.7647837828967856, recall=0.9254287177591733
####################
2022-06-03 08:40:51.454160 Training Epoch [023/030], [loss: 0.6587, dice: 0.9426, iou: 0.8956]
[Saving Checkpoint:] ./snapshots/BiRAFPN_4_new_bottleneck_v2/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
s

scores: dice=0.9182949429252409, miou=0.8705647395269536, precision=0.946392649715706, recall=0.9080438511769193
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9133509717910766, miou=0.8649976598205372, precision=0.9201237687507609, recall=0.9380909171598989
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8246072223978883, miou=0.7463866415372749, precision=0.8354310798465183, recall=0.8692354219562276
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9017226734952198, miou=0.8372673010654691, precision=0.8667472987918985, recall=0.9622051669594359
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.8244437994615557, miou=0.7485184735637523, precision=0.7893035515227121, recall=0.9108706550136385
####################
2022-06-03 09:12:12.862441 Training Epoch [030/030], [loss: 

In [14]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('logsRA_new_bottleneck_v2/lan5')

init_lr = 1e-4
batchsize = 8
trainsize_init = 384
clip = 0.5
num_epochs= 30
train_save = 'BiRAFPN_5_new_bottleneck_v2'

save_path = './snapshots/{}/'.format(train_save)
if not os.path.exists(save_path):
    os.makedirs(save_path, exist_ok=True)
else:
    print("Save path existed")
#     sys.exit(1)


log = pd.DataFrame(index=[], columns=[
    'epoch', 'lr', 'loss', 'dice', 'iou', 'val_loss', 'val_dice', 'val_iou'
])
train_img_paths = []
train_mask_paths = []
train_img_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/image/*')
train_mask_paths = glob('../dataset/scenario_4/all_datasets/TrainDataset/mask/*')
train_img_paths.sort()
train_mask_paths.sort()

train_dataset = Dataset(train_img_paths, train_mask_paths,transform =train_transform)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    pin_memory=True,
    drop_last=True
)

total_step = len(train_loader)
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                  compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth').cuda()

# ---- flops and params ----
params = model.parameters()
optimizer = torch.optim.Adam(params, init_lr)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                    T_max=len(train_loader)*num_epochs,
                                    eta_min=init_lr/1000)



start_epoch = 1

ckpt_path = ''
if ckpt_path != '':
    log = pd.read_csv(ckpt_path.replace('last.pth', 'log.csv'))
    checkpoint = torch.load(ckpt_path)
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    lr_scheduler.load_state_dict(checkpoint['scheduler'])
    optimizer.load_state_dict(checkpoint['optimizer'])

print("#"*20, f"Start Training", "#"*20)
for epoch in range(start_epoch, num_epochs+1):
    train_log = train(train_loader, model, optimizer, epoch, lr_scheduler)

    log_tmp = pd.Series([epoch, optimizer.param_groups[0]["lr"], 
            train_log['loss'].item(), train_log['dice'].item(), train_log['iou'].item(),  
    ], index=['epoch', 'lr', 'loss', 'dice', 'iou'])
    log = log.append(log_tmp, ignore_index=True)
    log.to_csv(f'./snapshots/{train_save}/log.csv', index=False)
    writer.add_scalar('training loss',
                            train_log['loss'].item(),
                            epoch )
    writer.add_scalar('training dice',
                            train_log['dice'].item(),
                            epoch )
    writer.add_scalar('training iou',
                            train_log['iou'].item(),
                            epoch )
    
    if epoch >= num_epochs-20:
        inference(model,writer,epoch)

2022-06-03 09:13:18,610 - mmseg - INFO - Use load_from_local loader
2022-06-03 09:13:18,682 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



#################### Start Training ####################
2022-06-03 09:16:42.538286 Training Epoch [001/030], [loss: 2.4620, dice: 0.7821, iou: 0.6497]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_v2/last.pth
2022-06-03 09:20:07.197795 Training Epoch [002/030], [loss: 1.5543, dice: 0.8416, iou: 0.7320]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_v2/last.pth
2022-06-03 09:23:32.140386 Training Epoch [003/030], [loss: 1.4406, dice: 0.8195, iou: 0.7065]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_v2/last.pth
2022-06-03 09:26:57.092929 Training Epoch [004/030], [loss: 1.3986, dice: 0.8349, iou: 0.7285]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_v2/last.pth
2022-06-03 09:30:21.743752 Training Epoch [005/030], [loss: 1.1350, dice: 0.8801, iou: 0.7932]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_v2/last.pth
2022-06-03 09:33:46.512416 Training Epoch [006/030], [loss: 1.0119, dice: 0.9034, iou: 0.8264]
[Saving Checkpoint:] ./

scores: dice=0.7957810528961546, miou=0.7164079584999303, precision=0.7991150648233998, recall=0.8427011060000139
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.895920766226505, miou=0.8224635420498968, precision=0.8407299624857778, recall=0.9753375905139141
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.8103682904418126, miou=0.7264203741193587, precision=0.7513279182297227, recall=0.9293195908430643
####################
2022-06-03 10:14:18.605026 Training Epoch [016/030], [loss: 0.7949, dice: 0.9319, iou: 0.8764]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_v2/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir
scores: dice=0.9256767807010767, miou=0.8727996521827122, precision=0.9452664968760449, recall=0.920878464465325
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB

scores: dice=0.9188308832122313, miou=0.8664419015297443, precision=0.9218559551340705, recall=0.9363618884791473
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.7966186448925386, miou=0.7171365905236126, precision=0.8240103429708151, recall=0.8300601802096446
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9008173110695998, miou=0.8345604461920406, precision=0.8708363136620844, recall=0.9427151049354571
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.8279400980327775, miou=0.7481035573015008, precision=0.7913366768487006, recall=0.9220582306758865
####################
2022-06-03 10:45:40.306223 Training Epoch [023/030], [loss: 0.6305, dice: 0.9536, iou: 0.9134]
[Saving Checkpoint:] ./snapshots/BiRAFPN_5_new_bottleneck_v2/last.pth
####################
../dataset/scenario_4/all_datasets//TestDataset/Kvasir
Dataset_name: Kvasir

scores: dice=0.926643257826244, miou=0.8764203356342689, precision=0.9480647948163576, recall=0.9226585926585911
../dataset/scenario_4/all_datasets//TestDataset/CVC-ClinicDB
Dataset_name: CVC-ClinicDB
scores: dice=0.9395886947143941, miou=0.894074151202363, precision=0.9344919305786795, recall=0.9538248579598031
../dataset/scenario_4/all_datasets//TestDataset/CVC-ColonDB
Dataset_name: CVC-ColonDB
scores: dice=0.8079144732606574, miou=0.7357083357937648, precision=0.8224612087883432, recall=0.8468131622426033
../dataset/scenario_4/all_datasets//TestDataset/CVC-300
Dataset_name: CVC-300
scores: dice=0.9079713449358838, miou=0.8425432966284389, precision=0.8772566440268084, recall=0.9570589941835428
../dataset/scenario_4/all_datasets//TestDataset/ETIS-LaribPolypDB
Dataset_name: ETIS-LaribPolypDB
scores: dice=0.8320197220013231, miou=0.7566351862512937, precision=0.8066528795415293, recall=0.9113279624918371
####################
2022-06-03 11:17:01.325326 Training Epoch [030/030], [loss: 0

In [5]:
model = Net2( backbone=dict(
                  type='mit_b3',
                  style='pytorch'), 
              decode_head=dict(
                  type='UPerHead',
                  in_channels=[64, 128, 320, 448],
                  in_index=[0, 1, 2, 3],
                  channels=128,
                  dropout_ratio=0.1,
                  num_classes=1,
                  norm_cfg=dict(type='BN', requires_grad=True),
                   compound_coef=4,
                  align_corners=False,
                  decoder_params=dict(embed_dim=768),
                  loss_decode=dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0)),
              neck=None,
             compound_coef=4,
              auxiliary_head=None,
              train_cfg=dict(),
              test_cfg=dict(mode='whole'),
              pretrained='pretrained/mit_b3.pth')
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
from pthflops import count_ops
print(count_parameters(model))
x = torch.rand(1,3,384,384)
#count_ops(model, x,mode='jit')
from fvcore.nn import FlopCountAnalysis
flops = FlopCountAnalysis(model, x)
flops.total()

2022-06-03 13:45:55,566 - mmseg - INFO - Use load_from_local loader
2022-06-03 13:45:55,659 - mmseg - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.weight, head.bias



93489990


/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:98: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
/home/s/thuannh/SegFormer_clean/mmseg/models/backbones/mix_transformer.py:104: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  kv

50090527836

In [4]:
!nvidia-smi


Fri Jun  3 13:45:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 50%   32C    P8    43W / 350W |     31MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
import cv2,random, glob
import matplotlib.pyplot as plt
files = glob.glob("../dataset/ISIC/traindataset/images/*")


In [33]:
for path in files:
    name = path.split("/")[-1]
    image = cv2.imread(path)
    image = cv2.resize(image,(384,384))
    cv2.imwrite("../dataset/ISIC/384/traindataset/images/"+name,image)


In [34]:
print(len(glob.glob("../dataset/ISIC/traindataset/images/*")),len(glob.glob("../dataset/ISIC/384/traindataset/images/*")))

1815 1815
